In [16]:
from satellite_analysis.graphs import massrelation as massrelation
from satellite_analysis.massrelations import stellarmass_relations
import yt
import glob
import numpy as np

import matplotlib.pyplot as plt
from satellite_analysis.massrelations import stellarmass_relations

#first find all of the halomass catalogs

#add argparse and make the out_dir if it doesnt exits
#output_dir = '/Users/user1/documents/GEN6/thesisimages/stellarmassrelation'
#halomass_dirs = glob.glob('/Users/user1/documents/GEN6/rockstarcomoving001/stellarmassrelation/VELA*')

output_dir = '/Users/user1/documents/GEN3/thesisimages/stellarmassrelation'
halomass_dirs = glob.glob('/Users/user1/documents/GEN3/rockstar_moresnaps/stellarmassrelation/VELA*')


#now, find the earliest and latest snapshots for each catalog
earliest_snap = []
latest_snap = []
for directory in halomass_dirs:
    all_snaps = glob.glob('%s/*.ascii' % directory)
    all_snaps.sort()
    earliest_snap.append(all_snaps[0])
    latest_snap.append(all_snaps[-1])

#earliest_snap = earliest_snap[0][-9:-6]
#latest_snap = latest_snap[-1][-9:-6]

#now make the list of all possible snapshots from the earlies and latest snap
#a_list = [int(earliest_snap) + 10*x for x in range((int(latest_snap) - int(earliest_snap))//10 + 1)]
#print(a_list)

a_list = [200, 250, 330, 400, 500]
count = 1

#now now I set up a nested loop to iterate over each a in my a_list in each of my VELA_dir halomass folder
for a in a_list:
    print('Generating dataset for a: %s' % a)
    
    central_ids, satellite_ids = [], []
    central_stellarmass010rvir, satellite_stellarmass010rvir, = [], []
    central_stellarmass015rvir, satellite_stellarmass015rvir, = [], []
    central_stellarmass020rvir, satellite_stellarmass020rvir, = [], []
    
    central_ratio010rvir, satellite_ratio010rvir = [], []
    central_ratio015rvir, satellite_ratio015rvir = [], []
    central_ratio020rvir, satellite_ratio020rvir = [], [] 
    
    central_all_mass_mvir, satellite_all_mass_mvir = [], []
    
    for halomass_dir in halomass_dirs:
        print('Trying file: %s/halomass%s.ascii' % (halomass_dir, a))
        try:
            readfile = open('%s/halomass%s.ascii' % (halomass_dir, a))
            lines = readfile.readlines()
            halomass_lines = []
            for line in lines:
                halomass_lines.append(line.split())
            del halomass_lines[0]
            for data_lists in halomass_lines:
                #filter out the satellites (pid =/= -1) from the central galaxies they orbit (pid = -1)
                if float(data_lists[1]) == -1:
                    central_ids.append(float(data_lists[0]))
                    #need the Mvir for the centrals and the Mpeak for the satellites
                    
                    central_stellarmass010rvir.append(float(data_lists[4]))
                    central_stellarmass015rvir.append(float(data_lists[5]))
                    central_stellarmass020rvir.append(float(data_lists[6]))
                    
                    #Mvir = darkmatter + stars + gas @ rvir
                    Mvir = float(data_lists[2]) + float(data_lists[7]) + float(data_lists[8])
                    central_all_mass_mvir.append(Mvir)
                    
                    central_ratio010rvir.append(float(data_lists[4]) / Mvir)
                    central_ratio015rvir.append(float(data_lists[5]) / Mvir)
                    central_ratio020rvir.append(float(data_lists[6]) / Mvir)
                    
                else:
                    satellite_ids.append(float(data_lists[0]))
                    #need the Mvir for the centrals and the Mpeak for the satellites
                    
                    satellite_stellarmass010rvir.append(float(data_lists[4]))
                    satellite_stellarmass015rvir.append(float(data_lists[5]))
                    satellite_stellarmass020rvir.append(float(data_lists[6]))
                    
                    Mpeak = float(data_lists[3]) + float(data_lists[7]) + float(data_lists[8])
                    satellite_all_mass_mvir.append(Mpeak)
                    
                    satellite_ratio010rvir.append(float(data_lists[4]) / Mpeak)
                    satellite_ratio015rvir.append(float(data_lists[5]) / Mpeak)
                    satellite_ratio020rvir.append(float(data_lists[6]) / Mpeak)
            readfile.close()
        except FileNotFoundError:
            print('No file: %s/halomass%s.ascii' % (halomass_dir, a))
            
    #generate the Behroozi and Puebla relations to plot
    x = np.linspace(10**7.8, 10**14, 3000)
    Behroozirelation = stellarmass_relations.BehrooziWechslerConroy2013(a/1000, (1/(a/1000) - 1), x)
    Pueblarelation = stellarmass_relations.PueblaPrimack2017(a/1000, (1/(a/1000) - 1), x)
    
    if a == 200:
        plt.figure(1, figsize=(20,30))
        #plt.subplot(3, 2, count)
        #plt.title('Scale Factor %s' % str(a/1000))
        #plt.xlabel('halomass')
        #plt.ylabel('massratio')
        #plt.xscale('log')
        #plt.yscale('log')
        #plt.xlim(10**(7.8), 10**(14))
        #plt.ylim(0.0, 0.10)
        #plt.plot(x, Behroozirelation[3], 'k', color='#CC4F1B')
        #plt.plot(x, Pueblarelation[3], 'k', color='#1B2ACC')
        #plt.scatter(central_all_mass_mvir, central_ratio010rvir, s = 1)
        #plt.scatter(satellite_all_mass_mvir, satellite_ratio010rvir, s = 1, c = 'r')
        #plt.fill_between(x, Behroozirelation[5], Behroozirelation[4], alpha=0.5, edgecolor='#CC4F1B', facecolor='#FF9848')
        #plt.fill_between(x, Pueblarelation[5], Pueblarelation[4], alpha=0.5, edgecolor='#1B2ACC', facecolor='#089FFF')
        #plt.legend(('Behroozi2013', 'Rodriguez-Puebla2017',))
        
        #count = count + 1
        
        plt.subplot(3, 2, count)
        plt.title('Redshift {}'.format(1/(a/1000)-1))
        plt.xlabel('halomass')
        plt.ylabel('stellarmass')
        plt.xscale('log')
        plt.yscale('log')
        plt.xlim(10**(7.8), 10**(14))
        plt.ylim(10**(-2), 10**(14))
        plt.plot(x, 10**Behroozirelation[0], 'k', color='#CC4F1B')
        plt.plot(x, 10**Pueblarelation[0], 'k', color='#1B2ACC')
        plt.scatter(central_all_mass_mvir, central_stellarmass010rvir, s = 1)
        plt.fill_between(x, 10**Behroozirelation[2], 10**Behroozirelation[1], alpha=0.5, edgecolor='#CC4F1B', facecolor='#FF9848')
        plt.fill_between(x, 10**Pueblarelation[2], 10**Pueblarelation[1], alpha=0.5, edgecolor='#1B2ACC', facecolor='#089FFF')
        #plt.scatter(satellite_all_mass_mvir, satellite_stellarmass010rvir, s = 1, c = 'r')
        plt.legend(('Behroozi2013', 'Rodriguez-Puebla2017', 'Host Galaxy'))
        
        count = count + 1
        
    else:
        #plt.subplot(3, 2, count)
        #plt.title('Scale Factor %s' % str(a/1000))
        #plt.xlabel('halomass')
        #plt.ylabel('massratio')
        #plt.xscale('log')
        #plt.yscale('log')
        #plt.xlim(10**(7.8), 10**(14))
        #plt.ylim(0.0, 0.10)
        #plt.plot(x, Behroozirelation[3], 'k', color='#CC4F1B')
        #plt.plot(x, Pueblarelation[3], 'k', color='#1B2ACC')
        #plt.scatter(central_all_mass_mvir, central_ratio010rvir, s = 1)
        #plt.scatter(satellite_all_mass_mvir, satellite_ratio010rvir, s = 1, c = 'r')
        #plt.fill_between(x, Behroozirelation[5], Behroozirelation[4], alpha=0.5, edgecolor='#CC4F1B', facecolor='#FF9848')
        #plt.fill_between(x, Pueblarelation[5], Pueblarelation[4], alpha=0.5, edgecolor='#1B2ACC', facecolor='#089FFF')
        #plt.legend(('Behroozi2013', 'Rodriguez-Puebla2017', 'Host Galaxies', 'Satellite Galaxies'))
        
        #count = count + 1
        
        plt.subplot(3, 2, count)
        plt.title('Redshift {}'.format(str(1/(a/1000)-1)[0:3]))
        plt.xlabel('halomass')
        plt.ylabel('stellarmass')
        plt.xscale('log')
        plt.yscale('log')
        plt.xlim(10**(7.8), 10**(14))
        plt.ylim(10**(-2), 10**(14))
        plt.plot(x, 10**Behroozirelation[0], 'k', color='#CC4F1B')
        plt.plot(x, 10**Pueblarelation[0], 'k', color='#1B2ACC')
        plt.scatter(central_all_mass_mvir, central_stellarmass010rvir, s = 1)
        plt.fill_between(x, 10**Behroozirelation[2], 10**Behroozirelation[1], alpha=0.5, edgecolor='#CC4F1B', facecolor='#FF9848')
        plt.fill_between(x, 10**Pueblarelation[2], 10**Pueblarelation[1], alpha=0.5, edgecolor='#1B2ACC', facecolor='#089FFF')
        #plt.scatter(satellite_all_mass_mvir, satellite_stellarmass010rvir, s = 1, c = 'r')
        plt.legend(('Behroozi2013', 'Rodriguez-Puebla2017', 'Host Galaxy'))
        
        count = count + 1
        
        
        
        

plt.savefig('%s/hostsandsatellites5x2alla.png' % output_dir, bbox_inches='tight')
plt.close()


Generating dataset for a: 200
Trying file: /Users/user1/documents/GEN3/rockstar_moresnaps/stellarmassrelation/VELA06/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/rockstar_moresnaps/stellarmassrelation/VELA07/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/rockstar_moresnaps/stellarmassrelation/VELA08/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/rockstar_moresnaps/stellarmassrelation/VELA09/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/rockstar_moresnaps/stellarmassrelation/VELA10/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/rockstar_moresnaps/stellarmassrelation/VELA12/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/rockstar_moresnaps/stellarmassrelation/VELA14/halomass200.ascii
Trying file: /Users/user1/documents/GEN3/rockstar_moresnaps/stellarmassrelation/VELA15/halomass200.ascii
Generating dataset for a: 250
Trying file: /Users/user1/documents/GEN3/rockstar_moresnaps/stellarmassrelation/VELA06/halomass250.a